### Import libraries

In [26]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras

### Extract labels for images

In [22]:
result_array = pd.read_csv('dataset/subsystem_1/Dataset_Subsystem_1.csv')

In [28]:
result_array

,ID,source,frame,camera_facing_side,gesture,palm_root_x,palm_root_y,palm_thumb_1_x,palm_thumb_1_y,palm_thumb_2_x,...,dorsal_ring_4_x,dorsal_ring_4_y,dorsal_pinky_1_x,dorsal_pinky_1_y,dorsal_pinky_2_x,dorsal_pinky_2_y,dorsal_pinky_3_x,dorsal_pinky_3_y,dorsal_pinky_4_x,dorsal_pinky_4_y
0,102,open_palm.webm,0,open,palm,279,369,189,332,137,...,0,0,0,0,0,0,0,0,0,0
1,102,open_palm.webm,1,open,palm,279,370,188,331,137,...,0,0,0,0,0,0,0,0,0,0
2,102,open_palm.webm,2,open,palm,279,370,187,331,137,...,0,0,0,0,0,0,0,0,0,0
3,102,open_palm.webm,3,open,palm,278,370,186,330,136,...,0,0,0,0,0,0,0,0,0,0
4,102,open_palm.webm,4,open,palm,278,371,185,329,136,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11991,723,three_fingers_dorsal.webm,197,three_fingers,dorsal,0,0,0,0,0,...,0,0,800,312,0,0,0,0,0,0
11992,723,three_fingers_dorsal.webm,198,three_fingers,dorsal,0,0,0,0,0,...,0,0,800,311,0,0,0,0,0,0
11993,723,three_fingers_dorsal.webm,199,three_fingers,dorsal,0,0,0,0,0,...,0,0,800,311,0,0,0,0,0,0
11994,723,three_fingers_dorsal.webm,200,three_fingers,dorsal,0,0,0,0,0,...,0,0,800,311,0,0,0,0,0,0


In [24]:
data_array = result_array.values[:,5:]

In [29]:
data_array.shape

(11996, 80)

### Extract frames from videos and transform them into images (do not run: to much data for the RAM)

In [18]:
train_folders = ['102'] #'159', '294', '441', '564', '576', '609','666','711','723'
images_array = []
gestures = ['open_palm','open_dorsal','fist_palm','fist_dorsal','three_fingers_palm','three_fingers_dorsal']
for folder in train_folders:
    for gesture in gestures:
        video_reader = cv2.VideoCapture("dataset/subsystem_1/videos/"+folder+"/"+gesture+".webm")
        ret, frame = video_reader.read()
        while ret:
            ret, frame = video_reader.read()
            if not ret:
                continue
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            images_array.append(gray_frame)
images_array = np.array(images_array)
images_array.shape

(1706, 480, 640)

### Create model to extract landmarks

In [27]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier 

class KerasClassifier_Patched(KerasClassifier):
    # bugfix: classifier doesn't declare that it is a classifier
    # in the Scikit learn API
    _estimator_type = "classifier"
    
    # bugfix: the current wrapper does not work with HotOne encoded
    # labels
    # this is only a fix in the specific case of this notebook,
    # not a general one
    def score(self, x, y, **kwargs):
        _, accuracy = self.model.evaluate(x,y, verbose=0, **kwargs)
        return accuracy

ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'; 'tensorflow.keras' is not a package

In [ ]:
def setupModel():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(480,640)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(80))
    model.add(tf.keras.layers.Softmax())

    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-5),
                  loss=tf.keras.losses.MeanSquaredError()
                 )
    return model

model = KerasClassifier_Patched(build_fn=setupModel,
                                epochs=5,
                                batch_size=50,
                                verbose=0 # do not print training progress to console
                               )

### Validate the model